<a href="https://colab.research.google.com/github/rodgpt/MAR_FUTURA/blob/main/NDSI/NDSI_Sites_Comparisson_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This code compares the NDSI and bio-anthro energy for different cases. It allows to set a date range

In [ ]:
#This cell mounts the google drive and install packages to be able to run the rest in R, because the whole runtime is in phyton

from google.colab import drive
drive.mount('/content/drive')
!pip install rpy2
%load_ext rpy2.ipython

#Prueba de cambio ariel v3

In [ ]:
%%R

install.packages("tuneR")
install.packages("seewave")
install.packages("dplyr")
install.packages("purrr")
install.packages("ggplot2")
install.packages("scales")
install.packages("lubridate")
install.packages("tidyr")
install.packages("soundecology")
install.packages("beepr")
install.packages("future")
install.packages("future.apply")

library(tuneR)
library(seewave)
library(dplyr)
library(purrr)
library(ggplot2)
library(scales)
library(lubridate)
library(tidyr)
library(soundecology)
library(beepr)
library(future)
library(future.apply)

In [ ]:
%%R

#For running locally (Rod)
#dirs_sites <- list(
#  "San Antonio 28" = "/Users/rod/Library/CloudStorage/GoogleDrive-royanedel@marfutura.org/Unidades compartidas/MAR FUTURA/Hydrophones/San Antonio/28",
#  "Ventanas 39" = "/Users/rod/Library/CloudStorage/GoogleDrive-royanedel@marfutura.org/Unidades compartidas/MAR FUTURA/Hydrophones/Ventanas/20-10-25/39/Untitled",
#  "Zapallar 32" = "/Users/rod/Library/CloudStorage/GoogleDrive-royanedel@marfutura.org/Unidades compartidas/MAR FUTURA/Hydrophones/Zapallar/20-10-25/32"
#)

#For running in Colab
dirs_sites <- list(
  "San Antonio 28" = "/content/drive/Shareddrives/MAR FUTURA/Hydrophones/San Antonio/28",
  "Ventanas 39" = "/content/drive/Shareddrives/MAR FUTURA/Hydrophones/Ventanas/20-10-25/39/Untitled",
  "Zapallar 32" = "/content/drive/Shareddrives/MAR FUTURA/Hydrophones/Zapallar/20-10-25/32",
  "Las Cruces 41" = "/content/drive/Shareddrives/MAR FUTURA/Hydrophones/LasCruces/20-10-25/41"
)

segment_sec <- 250
anthro_band <- c(1000, 2000)
bio_band    <- c(2000, 3000) #c(2000, 3000)
tz <- "UTC"
analysis_duration <- NA
files_per_folder <- NA
output_csv <- "ndsi_casestudies_results.csv"

start_date <- as.POSIXct("2025-10-21 00:00:00", tz = tz)
end_date   <- as.POSIXct("2025-11-05 23:59:59", tz = tz)

workers <- max(1, parallel::detectCores() - 1)
future::plan(future::multisession, workers = workers)

extract_datetime <- function(filename) {
  dt_str <- sub("^(?:ST_\\d+_)?(\\d{8}_\\d{6})\\.WAV$", "\\1", basename(filename), ignore.case = TRUE)
  as.POSIXct(dt_str, format = "%Y%m%d_%H%M%S", tz = tz)
}

# Cache FFT bin indices across segments when segment length + sample rate stay constant
.ndsi_cache <- new.env(parent = emptyenv())

calculate_ndsi <- function(wave_obj) {
  nd <- ndsi(
    wave_obj,
    anthro_min = anthro_band[1], anthro_max = anthro_band[2],
    bio_min    = bio_band[1],    bio_max    = bio_band[2]
  )

  samples <- wave_obj@left
  sample_rate <- wave_obj@samp.rate
  n <- length(samples)

  cache_key <- paste(n, sample_rate, anthro_band[1], anthro_band[2], bio_band[1], bio_band[2], sep = "|")
  cached <- .ndsi_cache[[cache_key]]

  if (is.null(cached)) {
    freqs_hz <- (0:(n/2 - 1)) * (sample_rate / as.double(n))
    anthro_indices <- which(freqs_hz >= anthro_band[1] & freqs_hz <= anthro_band[2])
    bio_indices <- which(freqs_hz >= bio_band[1] & freqs_hz <= bio_band[2])
    cached <- list(anthro_indices = anthro_indices, bio_indices = bio_indices)
    .ndsi_cache[[cache_key]] <- cached
  }

  fft_result <- fft(samples)
  power_spectrum <- Mod(fft_result[1:(n/2)])^2 / n^2

  anthro_energy <- sum(power_spectrum[cached$anthro_indices])
  bio_energy <- sum(power_spectrum[cached$bio_indices])

  list(ndsi = nd$ndsi_left, anthro_energy = anthro_energy, bio_energy = bio_energy)
}

process_site <- function(directory, label) {
  files <- list.files(directory, pattern = "\\.wav$", full.names = TRUE, recursive = TRUE, ignore.case = TRUE)
  message("Found ", length(files), " files in ", label, " (searching recursively, case-insensitive)")
  files <- sort(files)

  # Pre-compute datetimes from filenames and filter files to those near the date window
  if (length(files) == 0) {
    file_dt <- tibble(
      filepath = character(),
      start_dt = as.POSIXct(character(), tz = tz)
    )
  } else {
    file_dt <- purrr::map_df(files, ~tibble(
      filepath = .x,
      start_dt = extract_datetime(.x)
    ))
  }

  if (nrow(file_dt) > 0) { # Only filter if there are rows to filter
    if (!is.na(start_date)) {
      # keep files that start not earlier than (start_date - segment_sec) to allow boundary segments
      file_dt <- dplyr::filter(file_dt, start_dt >= (start_date - segment_sec))
    }
    if (!is.na(end_date)) {
      file_dt <- dplyr::filter(file_dt, start_dt <= end_date)
    }
  }

  file_dt <- dplyr::arrange(file_dt, start_dt)
  files <- file_dt$filepath

  if (!is.na(files_per_folder)) {
    files <- head(files, files_per_folder)
  }

  res_list <- future.apply::future_lapply(files, function(fp) {
    message("Reading: ", fp)
    start_dt <- extract_datetime(fp)

    wav <- tryCatch(readWave(fp), error = function(e) {
      warning("Skipping unreadable file: ", fp)
      return(NULL)
    })
    if (is.null(wav)) return(NULL)

    dur_sec <- length(wav@left) / wav@samp.rate
    max_start <- max(0, dur_sec - segment_sec)
    starts <- seq(0, max_start, by = segment_sec)

    out <- vector("list", length(starts))
    k <- 0L

    for (st in starts) {
      segment_time <- start_dt + st

      if (!is.na(start_date) && segment_time < start_date) {
        next
      }
      if (!is.na(end_date) && segment_time > end_date) {
        break
      }

      seg <- tryCatch(
        extractWave(wav, from = st, to = st + segment_sec, xunit = "time"),
        error = function(e) return(NULL)
      )
      if (is.null(seg)) next

      ndsi_res <- calculate_ndsi(seg)

      k <- k + 1L
      out[[k]] <- tibble(
        Site = label,
        Time = segment_time,
        NDSI = ndsi_res$ndsi,
        Anthro_Energy = ndsi_res$anthro_energy,
        Bio_Energy = ndsi_res$bio_energy
      )
    }

    if (k == 0L) return(NULL)
    dplyr::bind_rows(out[seq_len(k)])
  })

  dplyr::bind_rows(Filter(Negate(is.null), res_list))
}

anthro_tag <- paste0("anthro_", anthro_band[1], "-", anthro_band[2])
bio_tag <- paste0("bio_", bio_band[1], "-", bio_band[2])

date_tag <- if (!is.na(start_date) && !is.na(end_date)) {
  paste0(format(start_date, "%Y%m%d"), "-", format(end_date, "%Y%m%d"))
} else {
  "alltime"
}

output_dir <- "/content/drive/Shareddrives/MAR FUTURA/Hydrophones/NDSIResults"
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

all_results <- bind_rows(
  lapply(names(dirs_sites), function(label) {
    res <- tryCatch(
      process_site(dirs_sites[[label]], label),
      error = function(e) {
        warning("Site failed: ", label, " - ", conditionMessage(e))
        return(tibble(
          Site = character(),
          Time = as.POSIXct(character()),
          NDSI = numeric(),
          Anthro_Energy = numeric(),
          Bio_Energy = numeric()
        ))
      }
    )

    output_csv_site <- file.path(output_dir, paste0(
      "ndsi_", gsub("[^A-Za-z0-9]+", "_", tolower(label)), "_",
      date_tag, "_", anthro_tag, "_", bio_tag, ".csv"
    ))
    write.csv(res, output_csv_site, row.names = FALSE)
    message("Saved to: ", output_csv_site)

    res
  })
)

if (!exists("all_results") || nrow(all_results) == 0) {
  stop("No WAV files found in the provided directories and date range. Please verify the paths in `dirs_sites` and the date filters.")
}

output_csv_all <- file.path(output_dir, paste0(
  "ndsi_casestudies_", date_tag, "_", anthro_tag, "_", bio_tag, ".csv"
))
write.csv(all_results, output_csv_all, row.names = FALSE)
message("Saved to: ", output_csv_all)

summary_stats <- all_results %>%
  group_by(Site) %>%
  summarize(
    Segments  = n(),
    Mean_NDSI = mean(NDSI, na.rm = TRUE),
    SD_NDSI   = sd(NDSI, na.rm = TRUE),
    Mean_Anthro_Energy = mean(Anthro_Energy, na.rm = TRUE),
    SD_Anthro_Energy = sd(Anthro_Energy, na.rm = TRUE),
    Mean_Bio_Energy = mean(Bio_Energy, na.rm = TRUE),
    SD_Bio_Energy = sd(Bio_Energy, na.rm = TRUE)
  )
print(summary_stats)

plot_data <- all_results

plot_data$Site <- factor(plot_data$Site, levels = names(dirs_sites))

## NDSI time-series plot
p_ndsi <- ggplot(plot_data, aes(x = Time, y = NDSI, color = Site)) +
  geom_line(size = 0.8) +
  facet_wrap(~Site, ncol = 1, scales = "free_x") +
  scale_x_datetime(
    date_labels = "%d-%b %H:%M",
    date_breaks = "2 hour",
    expand = expansion(mult = c(0.01, 0.01))
  ) +
  labs(
    title = "NDSI Over Time for Case Study Sites",
    x = "Date-Time",
    y = "NDSI"
  ) +
  theme_minimal() +
  theme(
    legend.position = "none",
    plot.title = element_text(face = "bold", size = 14),
    axis.text.x = element_text(
      angle = 45,
      hjust = 1,
      vjust = 1,
      size = 6,
      margin = margin(t = 5)
    ),
    strip.text = element_text(face = "bold")
  )

print(p_ndsi)

## Anthropogenic energy time-series plot
p_anthro <- ggplot(plot_data, aes(x = Time, y = Anthro_Energy, color = Site)) +
  geom_line(size = 0.8) +
  facet_wrap(~Site, ncol = 1, scales = "free_x") +
  scale_x_datetime(
    date_labels = "%d-%b %H:%M",
    date_breaks = "2 hour",
    expand = expansion(mult = c(0.01, 0.01))
  ) +
  labs(
    title = "Anthropogenic Energy Over Time for Case Study Sites",
    x = "Date-Time",
    y = "Anthropogenic Energy (arbitrary units)"
  ) +
  theme_minimal() +
  theme(
    legend.position = "none",
    plot.title = element_text(face = "bold", size = 14),
    axis.text.x = element_text(
      angle = 45,
      hjust = 1,
      vjust = 1,
      size = 6,
      margin = margin(t = 5)
    ),
    strip.text = element_text(face = "bold")
  )

print(p_anthro)

## Biophonic energy time-series plot
p_bio <- ggplot(plot_data, aes(x = Time, y = Bio_Energy, color = Site)) +
  geom_line(size = 0.8) +
  facet_wrap(~Site, ncol = 1, scales = "free_x") +
  scale_x_datetime(
    date_labels = "%d-%b %H:%M",
    date_breaks = "2 hour",
    expand = expansion(mult = c(0.01, 0.01))
  ) +
  labs(
    title = "Biophonic Energy Over Time for Case Study Sites",
    x = "Date-Time",
    y = "Biophonic Energy (arbitrary units)"
  ) +
  theme_minimal() +
  theme(
    legend.position = "none",
    plot.title = element_text(face = "bold", size = 14),
    axis.text.x = element_text(
      angle = 45,
      hjust = 1,
      vjust = 1,
      size = 6,
      margin = margin(t = 5)
    ),
    strip.text = element_text(face = "bold")
  )

print(p_bio)

## NDSI quadrant table (percentage of time per site in each quadrant)
ndsi_quadrant_table <- plot_data %>%
  mutate(
    NDSI_Quadrant = case_when(
      NDSI >= 0.5  & NDSI <= 1   ~ "[0.5, 1]",
      NDSI >  0    & NDSI <  0.5 ~ "(0, 0.5)",
      NDSI >= -0.5 & NDSI <= 0   ~ "[-0.5, 0]",
      NDSI >= -1   & NDSI < -0.5 ~ "[-1, -0.5)",
      TRUE ~ NA_character_
    )
  ) %>%
  filter(!is.na(NDSI_Quadrant)) %>%
  group_by(Site, NDSI_Quadrant) %>%
  summarise(
    Segments = n(),
    .groups = "drop_last"
  ) %>%
  mutate(
    Total_Segments = sum(Segments),
    Percent_Time = 100 * Segments / Total_Segments
  ) %>%
  ungroup()

print(ndsi_quadrant_table)

beepr::beep(3)
